Note: ONNX not yet fully working due to protobuf conflict, etc.

For now, use `torch.save()` and `torch.load()`

But this is interesting:

https://blog.waya.ai/deploy-deep-machine-learning-in-production-the-pythonic-way-a17105f1540e

In [ ]:
#conda install -c conda-forge onnx
#python -m pip install caffe2

In [1]:
import torch
import onnx

model = onnx.load('novianty2018.onnx')
model

ir_version: 3
producer_name: "pytorch"
producer_version: "0.3"
graph {
  node {
    input: "1"
    output: "3"
    op_type: "Transpose"
    attribute {
      name: "perm"
      ints: 1
      ints: 0
      type: INTS
    }
    doc_string: "C:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\nn\\functional.py(1026): linear\nC:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\nn\\modules\\linear.py(55): forward\nC:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\nn\\modules\\module.py(465): _slow_forward\nC:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\nn\\modules\\module.py(475): __call__\nC:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\nn\\modules\\container.py(91): forward\nC:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\nn\\modules\\module.py(465): _slow_forward\nC:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\nn\\modules\\module.py(475): __call__\nC:\\ProgramData\\Anaconda3\\lib\\site-packages\\torch\\jit\\__init__.py(109): forward\nC:\\ProgramDa

In [2]:

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %Bias, T0, Td, T0×Td, Mw[FLOAT, 5]\n) initializers (\n  %1[FLOAT, 2x5]\n  %2[FLOAT, 2x2]\n) {\n  %3 = Transpose[perm = [1, 0]](%1)\n  %4 = MatMul(%Bias, T0, Td, T0×Td, Mw, %3)\n  %5 = Sigmoid(%4)\n  %6 = Transpose[perm = [1, 0]](%2)\n  %7 = MatMul(%5, %6)\n  %y0=Tsunami-Yes, y1=Tsunami-No = Sigmoid(%7)\n  return %y0=Tsunami-Yes, y1=Tsunami-No\n}'

In [3]:
import caffe2.python.onnx.backend as backend
import numpy as np

rep = backend.prepare(model, device="CUDA:0") # or "CPU"
# For the Caffe2 backend:
#     rep.predict_net is the Caffe2 protobuf for the network
#     rep.workspace is the Caffe2 workspace for the network
#       (see the class caffe2.python.onnx.backend.Workspace)
outputs = rep.run(np.random.randn(10, 3, 224, 224).astype(np.float32))
# To run networks with more than one input, pass a tuple
# rather than a single numpy ndarray.
print(outputs[0])


ModuleNotFoundError: No module named 'caffe2'